In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import Locator, MaxNLocator # added 
from tqdm import tqdm
from scipy import stats, interpolate
import yaml

from n3jet.general import FKSModelRun


Bad key "legend.title_fontsize" on line 22 in
/mt/home/jbullock/.config/matplotlib/stylelib/notebook.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution
Using TensorFlow backend.


## Set up

In [13]:
yaml_file = '/mt/home/jbullock/n3jet/n3jet/runs/runs/diphoton/4g2A/RAMBO/delta_grid/small_batch/fks_all_legs_all_pairs_new_sherpa_cuts_pdf_delta_00025.yaml'

In [14]:
save_dir = 'paper_plots/6_point/error_plots/unit_fks_RAMBO_100k_1.5M_delta_00025/'

In [15]:
save_fig = 'unit_error_plot'

In [16]:
grid_type = 'unit' # vegas or unit
legs = 6

## Data processing

In [17]:
test_fks_model = FKSModelRun.from_yaml(yaml_file, training=False)

/mt/home/jbullock/n3jet/n3jet/runs/runs/diphoton/4g2A/RAMBO/delta_grid/small_batch/fks_all_legs_all_pairs_new_sherpa_cuts_pdf_delta_00025.yaml


In [18]:
test_momenta, test_nj = test_fks_model.load_data()

############### Momenta loaded ###############
############### NJet loaded ###############
Training on 1000000 PS points


In [19]:
cut_momenta, near_momenta, cut_nj, near_nj, pairs, _, weights = test_fks_model.split_data(
    momenta = test_momenta, 
    nj = test_nj,
    return_weights = True
)

  5%|▍         | 3082/62496 [00:00<00:01, 30818.19it/s]

Pair 1 of 14


  5%|▌         | 3360/62496 [00:00<00:01, 33590.90it/s]

Pair 2 of 14


  5%|▌         | 3327/62496 [00:00<00:01, 33262.34it/s]

...

  6%|▌         | 3464/62496 [00:00<00:01, 34632.35it/s]

Pair 13 of 14


  5%|▌         | 3202/62496 [00:00<00:01, 32019.95it/s]

Pair 14 of 14


100%|██████████| 62496/62496 [00:01<00:00, 32570.17it/s]


In [20]:
cut_nj = np.array(cut_nj)
near_nj = np.array(near_nj)
total_nj = np.concatenate((cut_nj, near_nj))
total_momenta = np.concatenate((cut_momenta, near_momenta))

In [21]:
assert len(total_nj) == len(test_momenta)
assert len(total_momenta) == len(test_momenta)

test_points = len(test_momenta)
model_dir_new = test_fks_model.model_base_dir + test_fks_model.model_dir

In [22]:
nn_cut_all = []
nn_near_all = []
for t in tqdm(range(test_fks_model.training_reruns)):
    nn_cut = np.load(model_dir_new + '_{}/pred_cut_{}.npy'.format(t,test_points), allow_pickle=True)
    nn_near = np.load(model_dir_new + '_{}/pred_near_{}.npy'.format(t,test_points), allow_pickle=True)
    nn_cut_all.append(nn_cut)
    nn_near_all.append(nn_near)

nn_cut_std = np.std(nn_cut_all, ddof=1, axis=0)
nn_near_std = np.std(nn_near_all, ddof=1, axis=0)
    
nn_cut_mean = np.mean(nn_cut_all, axis=0)
nn_near_mean = np.mean(nn_near_all, axis=0)
    
assert len(nn_cut_mean) == len(cut_nj)
assert len(nn_near_mean) == len(near_nj)
    
total_nn = np.concatenate((nn_cut_mean, nn_near_mean))
total_nn_std = np.concatenate((nn_cut_std, nn_near_std))

assert len(total_nn) == test_points
assert len(total_nn_std) == test_points

100%|██████████| 20/20 [00:02<00:00,  9.02it/s]


AssertionError: 

## Error plots

In [ ]:
Delta = total_nn/total_nj
positive_locs = np.where(Delta >= 0.)
Delta = Delta[positive_locs]
    
log_delta = np.log(Delta)

In [ ]:
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

plt.hist(log_delta, bins=200, color='green', alpha = 0.5)
plt.xlim((-2.5,2.5))

plt.tick_params(axis='x', labelsize=15, direction = 'in', top = True)
plt.tick_params(axis='y', labelsize=15, direction = 'in', right = True)
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))

plt.ylabel('Frequency', fontsize=17, labelpad=10)
plt.xlabel(r'ln($\Delta$)', fontsize=17, labelpad=10)

if grid_type == 'vegas':
    annotation_string = "VEGAS grid \n"
elif grid_type == 'unit':
    annotation_string = "Unit grid \n"
if legs == 5:
    annotation_string += r"$gg\to \gamma\gamma g$"
elif legs == 6:
    annotation_string += r"$gg\to \gamma\gamma gg$"
annotation_string += "\n"
annotation_string += r"$\Delta = \frac{\mathrm{NN}}{\mathrm{NJet}}$"
    
plt.text(-2, 1e5, annotation_string, size=17,
         va="baseline", ha="left", multialignment="left", linespacing = 1.6)

plt.savefig(save_dir + save_fig + '.png', dpi=150, bbox_inches='tight')
plt.show()